In [1]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import pickle
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

<h2>GET ALL DATA FROM UNITED NATIONS FOOD AND AGRICULTURE ORGANIZATION CSVs</h2>

In [2]:
#Load the crop data
ag_production_filepath = Path('data','production_breakdown.csv')
ag_df = pd.read_csv(ag_production_filepath)
# Load the country code data
cc_filepath = Path('data', 'country-codes.csv')
cc_df = pd.read_csv(cc_filepath)
# Load exchange rate data
xr_filepath = Path('data', 'exchange_rates.csv')
xr_df = pd.read_csv(xr_filepath)
#Load the fertilizer data
ft_filepath = Path('data', 'fertilizer_total.csv')
ft_df = pd.read_csv(ft_filepath)
ftn_filepath = Path('data', 'fertilizer_by_nutrients.csv')
ftn_df = pd.read_csv(ftn_filepath)
#Load the pesticide data
pt_filepath = Path('data', 'pesticides.csv')
pt_df = pd.read_csv(pt_filepath)

In [22]:
#create a new dataframe to house all the data from the csvs
df = pd.DataFrame()

In [3]:
# Get a List of All Countries
countries_list = ag_df['Area'].drop_duplicates().sort_values()
# Get a List of All Years
years_list = ag_df['Year'].drop_duplicates().sort_values()
# Get a List of All Elements in the Production Data
elements_list = ag_df['Element'].drop_duplicates().sort_values()

In [4]:
years_series = pd.Series(years_list)
country_series = pd.Series(countries_list)
elements_series = pd.Series(elements_list)

In [5]:
countries_set = []
for index, value in years_series.items():
    for c_index, c_value in country_series.items():
            countries_set.append(c_value)

In [6]:
years_set = []
for y_index, y_value in years_series.items():
    for c_index, c_value in country_series.items():
        years_set.append(y_value)

In [155]:
df['Country'] = countries_set

In [156]:
df['Year'] = years_set

In [ ]:
#set the country codes - needed for exchange rate
for index, row in cc_df.iterrows():
    df.loc[df['Country'] == row['Country'], 'Country Code'] = row['Country_Code']

In [ ]:
#set the exchange rate - needed for Price USD
for index, row in xr_df.iterrows():
    df.loc[(df["Year"] == row["TIME"]) & (df["Country Code"] == row["LOCATION"]), 'Exchange Rate'] = row['Value']

In [26]:
#set up pesticides
for year in range(1990,2018):
    for country in country_series:
        if(pt_df.loc[(pt_df['Year'] == year) & (pt_df['Area'] == country), "Value"].values.size > 0):
            df.loc[(df['Year'] == year) & (df['Country'] == country), "Pesticides"] = pt_df.loc[(pt_df['Year'] == year) & (pt_df['Area'] == country), "Value"].values[0]            

In [19]:
#set up fertilizer
for year in range(df['Year'].min(),(df['Year'].max()+1)):
    for country in country_series:
        if(ft_df.loc[(ft_df['Year'] == year) & (ft_df['Country'] == country), "Value"].values.size > 0):
                df.loc[(df['Year'] == year) & (df['Country'] == country), "Fertilizer"] = ft_df.loc[(ft_df['Year'] == year) & (ft_df['Country'] == country), "Value"].values[0]

In [ ]:
def element_data(element):
    for year in range(df['Year'].min(),(df['Year'].max()+1)):
        for country in country_series:
            if(ag_df.loc[(ag_df['Year'] == year) & (ag_df['Area'] == country) & (ag_df['Element'] == element), "Value"].values.size > 0):
                df.loc[(df['Year'] == year) & (df['Country'] == country), element] = ag_df.loc[(ag_df['Year'] == year) & (ag_df['Area'] == country) & (ag_df['Element'] == element), "Value"].values[0]

In [ ]:
for index, value in elements_series.items():
    df[value] = 0 

In [ ]:
#set domestic supply to allow percentage columns 
df["Domestic Supply"] = df["Production"] + df["Import Quantity"] - df["Export Quantity"] + df["Stock Variation"]

In [14]:
ftn_group = ftn_df.groupby(["Area", "Year"])["Value"].sum()
for index, value in ftn_group.items():
    df.loc[(df['Year'] == index[1]) & (df['Country'] == index[0]), "Fertilizer"] = value

In [172]:
# This gets most of the features/column info
for e_index, e_value in elements_series.items():
    element_data(e_value)

In [ ]:
df["Price USD"] = df['Producer Price (LCU/tonne)'] * df['Exchange Rate']

In [7]:
df['food_supply_percentage'] = df['Food']/df['Domestic Supply']*100
df['feed_supply_percentage'] = df['Feed']/df['Domestic Supply']*100
df['other_use_supply_percentage'] = df['Other uses (non-food)']/df['Domestic Supply']*100

# Exports need to be a percentage of domestic production, as you can only export what you actually grew
df['export_supply_percentage'] = df['Export Quantity']/df['Production']*100 
df['import_supply_percentage'] = df['Import Quantity']/df['Domestic Supply']*100


In [ ]:
df.loc[(df['Exchange Rate'].notnull()) & (df['Producer Price (LCU/tonne)'].notnull())]

In [8]:
# Create Pickle Files
df.to_pickle("df.pkl")

<h2>Use Pickle File To Populate DataFrame</h2>

In [3]:
with open("df.pkl", "rb") as file:
    df = pickle.load(file)

In [6]:
df.tail()

,Country,Year,Country Code,Exchange Rate,Producer Price (LCU/tonne),Price USD,Area harvested,Export Quantity,Feed,Food,...,Stock Variation,Yield,Domestic Supply,Pesticides,Fertilizer,food_supply_percentage,feed_supply_percentage,export_supply_percentage,other_use_supply_percentage,import_supply_percentage
14089,Western Sahara,2018,ESH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14090,Yemen,2018,YEM,NaN,NaN,NaN,37231.0,NaN,NaN,NaN,...,NaN,11546.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14091,Yugoslav SFR,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14092,Zambia,2018,ZMB,NaN,1856.8,NaN,1086006.0,NaN,NaN,NaN,...,NaN,22052.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14093,Zimbabwe,2018,ZWE,NaN,NaN,NaN,1191425.0,NaN,NaN,NaN,...,NaN,6131.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
